Configurar la conexión con la API de Power BI

In [1]:
#Librerías
from powerbiclient import Report, models
from powerbiclient.authentication import DeviceCodeLoginAuthentication
from datetime import datetime as dt
import requests, os, sys
from time import sleep
import pandas as pd
#Modulos DEV
my_lib_path = os.path.abspath('//135.208.36.251/Op_Intelligence/DOCUMENTACION/Scripts/Bots')
sys.path.append(my_lib_path)
import Libraries.db_connection as dbConn
#Variables
today = dt.today()
#Autenticación
device_auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FGZEL5CGD to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


Generar el programa para la creación del Dataframe

In [3]:
from time import sleep
class api_pbi_get_info:
    #conexión a servidor
    def conn_blue(self):
        try:
            conn_url, engine_blue, conn_blue, cursor_blue = dbConn.get_ETL_OIEngineBlue_Connection()
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Conexión éxitosa OI Bue Server')
            return engine_blue, conn_blue, cursor_blue
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on conn_blue: ' +  repr(error)) 
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on conn_blue: ' +  repr(error))
    #Dataframes vacios
    def empty_df(self):
        try:
            #columnas para df usuarios
            columns_user = ['id','tipo_acceso','nombre','tipo_usuario','id_workspace','nombre_workspace','mtd_date']
            df_users = pd.DataFrame(columns = columns_user)
            #columnas para df reportes
            columns_report = ['id','tipo_objeto','nombre','web_url','id_workspace','nombre_workspace','embebed_sharepoint','id_dataset','mtd_date']
            df_reports = pd.DataFrame(columns = columns_report)
            #columnas para df datasets
            columns_dataset = ['id','tipo_objeto','nombre','nombre_workspace','configurado_por','on_gateway','store_mode','fecha_creacion','mtd_date']
            df_datasets = pd.DataFrame(columns = columns_dataset)
            #columnas para df paginas
            columns_pages = ['id','nombre','orden','nombre_workspace','nombre_reporte','id_reporte','mtd_date']
            #df_pages = pd.DataFrame(columns = columns_pages)
            #return
            return columns_user, columns_report, columns_dataset, columns_pages, df_users, df_reports, df_datasets#, df_pages
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on empty_df: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on empty_df: ' +  repr(error))
    #Configuración de token y espacios a escanear
    def configuracion(self):
        try:
            #Token
            access_token = device_auth.get_access_token()
            #Espacios a escanear
            workspace_dict = {
                'ACE': '8fcb8600-1a48-4d76-ab46-c54dcd651ae6'
                ,'IO_CEC': '1030401e-ca90-450e-8372-059f952e1486'
                }
            #tablas a trabajar
            tbl_users = 'tbl_api_pbi_usuarios'
            tbl_reports = 'tbl_api_pbi_reportes'
            tbl_dataset = 'tbl_api_pbi_datasets'
            tbl_pages = 'tbl_api_pbi_pages'
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Se ha definido el token y espacios de trabajo')
            return access_token, workspace_dict, tbl_users, tbl_reports, tbl_dataset, tbl_pages
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on configuracion: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on configuracion: ' +  repr(error))
    #Obtener los usuarios a trabajar
    def get_api_data(self, access_token, workspace_id, workspace_name):
        try:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Escaneando: ' +  workspace_name)
            url_users = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspace_id + '/users'
            url_reports = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspace_id + '/reports'
            url_datasets = 'https://api.powerbi.com/v1.0/myorg/groups/' + workspace_id + '/datasets'
            #header global
            headers = {
                'Authorization': f'Bearer {access_token}',
                'Content-type': 'application/json'
            }
            #Autorizacion users
            response_users = requests.get(url_users, headers = headers)
            users = response_users.json()['value']
            #Autorizacion repors
            response_reports = requests.get(url_reports, headers = headers)
            reports = response_reports.json()['value']
            #Autorizacion datasets
            response_datasets = requests.get(url_datasets, headers = headers)
            datasets = response_datasets.json()['value']
            #Proceso terminado
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Los usuarios y reportes fueron obtenidos correctamente')
            return users, reports, datasets
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on get_api_data: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on get_api_data: ' +  repr(error))
    #Genera el dataframe de usuarios
    def dataframe_users(self, columns_user, users, workspace_id, workspace_name, df_users):
        try:
            for r in users:
                #Listado de datos
                data_list = [[r['identifier'], r['groupUserAccessRight'], r['displayName'], r['principalType'], workspace_id, workspace_name, today]]
                #Transformaciones a dataframe
                df_report = pd.DataFrame(data_list, columns = columns_user)
                df_users = pd.concat([df_users, df_report])
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Dataframe de usuarios listo para carga')
            return df_users
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_users: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_users: ' +  repr(error))
    #Genera el dataframe de reportes
    def dataframe_reports(self, columns_report, reports, workspace_name, df_reports):
        try:
            for r in reports:
                #Transformaciones
                embebed = r['embedUrl'][:r['embedUrl'].find('&')]
                #Listado de datos
                data_list = [[r['id'], r['reportType'], r['name'], r['webUrl'], r['datasetWorkspaceId'], workspace_name
                              ,embebed, r['datasetId'], today]]
                #Transformaciones a dataframe
                df_report = pd.DataFrame(data_list, columns = columns_report)
                df_reports = pd.concat([df_reports, df_report])
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Dataframe de reportes listo para carga')
            return df_reports
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_reports: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_reports: ' +  repr(error))
    #Genera el dataframe de reportes
    def dataframe_datasets(self, columns_dataset, datasets, workspace_name, df_datasets):
        try:
            for r in datasets:
                #transformaciones
                fecha_creacion = dt.strptime(r['createdDate'], "%Y-%m-%dT%H:%M:%S.%fZ")
                #Listado de datos
                data_list = [[r['id'], 'PowerBIDataset', r['name'], workspace_name, r['configuredBy'], str(r['isOnPremGatewayRequired'])
                              , r['targetStorageMode'], fecha_creacion, today]]
                #Transformaciones a dataframe
                df_dataset = pd.DataFrame(data_list, columns = columns_dataset)
                df_datasets = pd.concat([df_datasets, df_dataset])
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Dataframe de datasets listo para carga')
            return df_datasets
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_datasets: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_datasets: ' +  repr(error))
    #Genera el dataframe de hojas
    def dataframe_pages(self, access_token, workspace_id, workspace_name, reports, columns_pages, df_pages):
        try:
            for r in reports:
                sleep(5)
                reporte = r['id']
                url_pages = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/reports/{reporte}/pages/'
                headers = {'Authorization': f'Bearer {access_token}', 'Content-type': 'application/json'}
                response_pages = requests.get(url_pages, headers = headers)
                pages = response_pages.json()['value']
                for p in pages:
                    data_list = [[p['name'], p['displayName'], p['order'], workspace_name, r['name'], r['id'], today]]
                    df_page = pd.DataFrame(data_list, columns = columns_pages)
                    df_pages = pd.concat([df_pages, df_page])
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Dataframe de páginas listo para carga')
            return df_pages
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_pages: ' +  repr(error))
            raise ValueError(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on dataframe_pages: ' +  repr(error))
    #Limpia usuarios
    def clean_sql(self, conn, cursor, tabla):
        try:
            query = 'DELETE FROM io_internal.' + tabla
            cursor.execute(query)
            conn.commit()
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Se limpió la tabla io_internal.' + tabla)
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on clean_sql: ' +  repr(error)) 
            raise ValueError('Error on clean_sql')
    #Carga a SQL
    def load_to_sql(self, df_users, engine, conn, tabla):
        try:
            df_users.to_sql(tabla, engine, index=False, if_exists="append", schema="io_internal")
            conn.commit()
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Carga completada en io_internal.' + tabla)
        except Exception as error:
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ': Error on load_to_sql: ' +  repr(error)) 
            raise ValueError('Error on load_to_sql')
    #Proceso completo
    def load(self, proc):
        try:
            ''''''
            engine_blue, conn_blue, cursor_blue = api_pbi_get_info.conn_blue(self)
            columns_user, columns_report, columns_dataset, columns_pages, df_users, df_reports, df_datasets = api_pbi_get_info.empty_df(self)
            access_token, workspace_dict, tbl_users, tbl_reports, tbl_dataset, tbl_pages = api_pbi_get_info.configuracion(self)
            for space in workspace_dict:
                workspace_name = space
                workspace_id = workspace_dict[space]
                users, reports, datasets = api_pbi_get_info.get_api_data(self, access_token, workspace_id, workspace_name)
                #prueba get pages
                #generar dataframe de usuarios
                df_users = api_pbi_get_info.dataframe_users(self, columns_user, users, workspace_id, workspace_name, df_users)
                df_reports = api_pbi_get_info.dataframe_reports(self, columns_report, reports, workspace_name, df_reports)
                df_datasets = api_pbi_get_info.dataframe_datasets(self, columns_dataset, datasets, workspace_name, df_datasets)
                #genera dataframe de paginas
                #df_pages = api_pbi_get_info.dataframe_pages(self, access_token, workspace_id, workspace_name, reports, columns_pages, df_pages)
            #SQL usuarios
            sql_dict = {tbl_users: df_users, tbl_reports: df_reports, tbl_dataset: df_datasets}#, tbl_pages: df_pages}
            for table in sql_dict:
                dataframe = sql_dict[table]
                #Eliminacion y carga
                api_pbi_get_info.clean_sql(self, conn_blue, cursor_blue, table)
                api_pbi_get_info.load_to_sql(self, dataframe, engine_blue, conn_blue, table)
        except Exception as error:
            msg_proc = 'Se detecto ese error:: ' + repr(error)
        else:
            msg_proc =  'Proceso completado sin errores'
        finally:
            print(msg_proc)
            print(dt.strftime(dt.now(), '%d/%m/%Y %H:%M:%S') + ": The '" + proc + "' is finished")
    
def main():
    runObj = api_pbi_get_info()
    run_process = 'api_pbi_get_info'
    result = runObj.load(run_process)
main()
'''
'''

09/09/2024 12:58:31: Conexión éxitosa OI Bue Server
09/09/2024 12:58:31: Se ha definido el token y espacios de trabajo
09/09/2024 12:58:31: Escaneando: ACE
09/09/2024 12:58:34: Los usuarios y reportes fueron obtenidos correctamente
09/09/2024 12:58:35: Dataframe de usuarios listo para carga
09/09/2024 12:58:35: Dataframe de reportes listo para carga
09/09/2024 12:58:35: Dataframe de datasets listo para carga
09/09/2024 12:58:35: Escaneando: IO_CEC
09/09/2024 12:58:38: Los usuarios y reportes fueron obtenidos correctamente
09/09/2024 12:58:38: Dataframe de usuarios listo para carga
09/09/2024 12:58:38: Dataframe de reportes listo para carga
09/09/2024 12:58:38: Dataframe de datasets listo para carga
09/09/2024 12:58:38: Se limpió la tabla io_internal.tbl_api_pbi_usuarios
09/09/2024 13:04:54: Carga completada en io_internal.tbl_api_pbi_usuarios
09/09/2024 13:04:54: Se limpió la tabla io_internal.tbl_api_pbi_reportes
09/09/2024 13:05:25: Carga completada en io_internal.tbl_api_pbi_reporte

'\n'